In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import shutil

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
tf.keras.backend.clear_session()
produk_kopi = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR0m9FjHszfjZh1ei--UGZnUivJQSoU2-pRdGrWNHaXtqBWneZNbEdoZg_cOZ87gJTDjJMtGcGphED_/pub?gid=165615979&single=true&output=csv')
konsumen_kopi = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR0m9FjHszfjZh1ei--UGZnUivJQSoU2-pRdGrWNHaXtqBWneZNbEdoZg_cOZ87gJTDjJMtGcGphED_/pub?gid=2012083633&single=true&output=csv')
data_kopi = produk_kopi
data = konsumen_kopi

In [ ]:
data

,idUser,namaUser,idProduk,jenisKelamin,namaProduk,kategoriProduk,skorAroma,skorAsam,asalUser
0,K039,Hadi Susanto,P001,Laki-laki,Kopi Arabika Sumatera,Arabika,8.9,2.9,Bengkulu
1,K090,Eko Santoso,P001,Laki-laki,Kopi Arabika Sumatera,Arabika,8.9,2.9,Lampung
2,K127,Iskandar Hassan,P001,Laki-laki,Kopi Arabika Sumatera,Arabika,8.9,2.9,Kepulauan Bangka Belitung
3,K209,Azman bin Ismail,P001,Laki-laki,Kopi Arabika Sumatera,Arabika,8.9,2.9,Kepulauan Riau
4,K277,Ibrahim Hassan,P001,Laki-laki,Kopi Arabika Sumatera,Arabika,8.9,2.9,Jakarta
...,...,...,...,...,...,...,...,...,...
679,K386,Hakim Yusof,P052,Laki-laki,Kopi Robusta Sumatra,Single Origin,6.7,5.7,Bali
680,K429,Hakim Azman,P052,Laki-laki,Kopi Robusta Sumatra,Single Origin,6.7,5.7,Nusa Tenggara Barat
681,K509,Azizah Ibrahim,P052,Perempuan,Kopi Robusta Sumatra,Single Origin,6.7,5.7,Nusa Tenggara Timur
682,K598,Maya Fitri,P052,Perempuan,Kopi Robusta Sumatra,Single Origin,6.7,5.7,Kalimantan Barat


In [ ]:
data = konsumen_kopi[['idProduk', 'kategoriProduk', 'skorAroma', 'skorAsam']]
data = data.drop_duplicates()

In [ ]:
label_encoder = LabelEncoder()
data['kategoriProduk'] = label_encoder.fit_transform(data['kategoriProduk'])
data['idProduk'] = label_encoder.fit_transform(data['idProduk'])

In [ ]:
data = data.apply(pd.to_numeric)

In [ ]:
df = data
df.dtypes

idProduk            int64
kategoriProduk      int64
skorAroma         float64
skorAsam          float64
dtype: object

In [ ]:
  from sklearn.preprocessing import MinMaxScaler

  scaler = MinMaxScaler()
  columns_to_normalize = ['skorAroma', 'skorAsam']
  df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
  df

,idProduk,kategoriProduk,skorAroma,skorAsam
0,0,0,0.870130,0.104478
13,1,6,0.519481,0.761194
27,2,0,0.909091,0.731343
40,3,5,0.675325,0.895522
52,4,1,0.389610,0.582090
62,5,0,0.038961,0.970149
78,6,0,0.792208,0.582090
92,7,3,0.259740,0.626866
104,8,0,0.545455,0.507463
119,9,2,0.116883,0.910448


In [ ]:
df.to_csv('kopi.csv')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

# Load data
data = df

features = data[['skorAroma', 'skorAsam', 'kategoriProduk']].values

similarity_matrix = tf.matmul(features, features, transpose_b=True)

top_3_similar = tf.argsort(similarity_matrix, axis=-1, direction='DESCENDING')[:, 1:4]

item_id = 0
top_3_similar_items = top_3_similar[item_id].numpy()
print(f'Top 3 similar items for item with idProduk = {item_id}: {top_3_similar_items}')


Top 3 similar items for item with idProduk = 0: [49  2 19]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

tf.keras.backend.clear_session()

data = df

features = data[['skorAroma', 'skorAsam','kategoriProduk']].values

inputs = keras.Input(shape=(features.shape[1],))
x = layers.Dense(16, activation='relu')(inputs)
outputs = layers.Dense(3)(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='mse')

model.fit(features, features, epochs=250)

model.save('recommender_model.h5')

Epoch 1/250
2/2 [==============================] - 2s 16ms/step - loss: 4.0670
Epoch 2/250
2/2 [==============================] - 0s 9ms/step - loss: 3.9262
Epoch 3/250
2/2 [==============================] - 0s 6ms/step - loss: 3.7921
Epoch 4/250
2/2 [==============================] - 0s 12ms/step - loss: 3.6618
Epoch 5/250
2/2 [==============================] - 0s 6ms/step - loss: 3.5350
Epoch 6/250
2/2 [==============================] - 0s 7ms/step - loss: 3.4060
Epoch 7/250
2/2 [==============================] - 0s 7ms/step - loss: 3.2806
Epoch 8/250
2/2 [==============================] - 0s 15ms/step - loss: 3.1609
Epoch 9/250
2/2 [==============================] - 0s 14ms/step - loss: 3.0292
Epoch 10/250
2/2 [==============================] - 0s 6ms/step - loss: 2.9338
Epoch 11/250
2/2 [==============================] - 0s 14ms/step - loss: 2.8151
Epoch 12/250
2/2 [==============================] - 0s 15ms/step - loss: 2.7091
Epoch 13/250
2/2 [==============================] - 0s 

In [ ]:
# Compute similarity between items using their learned representations
item_representations = model.predict(features)
similarity_matrix = tf.matmul(item_representations,
                              item_representations,
                              transpose_b=True)

# Get top 5 similar items for each item
top_5_similar = tf.argsort(similarity_matrix,
                           axis=-1,
                           direction='DESCENDING')[:, 1:6]

# Example: Get top 5 similar items for item with idProduk = 0
item_id = 1
top_5_similar_items = top_5_similar[item_id].numpy()
print(f'Top 5 similar items for item with idProduk = {item_id}: {top_5_similar_items}')

2/2 [==============================] - 0s 4ms/step
Top 5 similar items for item with idProduk = 1: [49 12 21 25 29]
